This notebook is designed to run in a IBM Watson Studio Apache Spark runtime. In case you are running it in an  IBM Watson Studio standard runtime or outside Watson Studio, we install Apache Spark in local mode for test purposes only. Please don't use it in production.

In [ ]:
# !pip install --upgrade pip

In [ ]:
# if not ('sc' in locals() or 'sc' in globals()):
#     print('It seems you are note running in a IBM Watson Studio Apache Spark Notebook. You might be running in a IBM Watson Studio Default Runtime or outside IBM Waston Studio. Therefore installing local Apache Spark environment for you. Please do not use in Production')
    
#     from pip import main
#     main(['install', 'pyspark==2.4.5'])
    
#     from pyspark import SparkContext, SparkConf
#     from pyspark.sql import SparkSession

#     sc = SparkContext.getOrCreate(SparkConf().setMaster("local[*]"))
    
#     spark = SparkSession \
#         .builder \
#         .getOrCreate()


In [1]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext, SparkSession
from pyspark.sql.types import StructType, StructField, DoubleType, IntegerType, StringType

sc = SparkContext.getOrCreate(SparkConf().setMaster("local[*]"))

from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .getOrCreate()

spark

In case you want to learn how ETL is done, please run the following notebook first and update the file name below accordingly

https://github.com/IBM/coursera/blob/master/coursera_ml/a2_w1_s3_ETL.ipynb

In [2]:
# delete files from previous runs
# !rm -f hmp.parquet*

# download the file containing the data in PARQUET format
# !wget https://github.com/IBM/coursera/raw/master/hmp.parquet
    
# create a dataframe out of it
df = spark.read.parquet('hmp.parquet')

# register a corresponding query table
df.createOrReplaceTempView('df')

In [3]:
from pyspark.ml.feature import StringIndexer

indexer = StringIndexer(inputCol="class", outputCol="classIndex")
indexed = indexer.fit(df).transform(df)

indexed.show()
indexed.select('classIndex').distinct().show()

+---+---+---+--------------------+-----------+----------+
|  x|  y|  z|              source|      class|classIndex|
+---+---+---+--------------------+-----------+----------+
| 22| 49| 35|Accelerometer-201...|Brush_teeth|       6.0|
| 22| 49| 35|Accelerometer-201...|Brush_teeth|       6.0|
| 22| 52| 35|Accelerometer-201...|Brush_teeth|       6.0|
| 22| 52| 35|Accelerometer-201...|Brush_teeth|       6.0|
| 21| 52| 34|Accelerometer-201...|Brush_teeth|       6.0|
| 22| 51| 34|Accelerometer-201...|Brush_teeth|       6.0|
| 20| 50| 35|Accelerometer-201...|Brush_teeth|       6.0|
| 22| 52| 34|Accelerometer-201...|Brush_teeth|       6.0|
| 22| 50| 34|Accelerometer-201...|Brush_teeth|       6.0|
| 22| 51| 35|Accelerometer-201...|Brush_teeth|       6.0|
| 21| 51| 33|Accelerometer-201...|Brush_teeth|       6.0|
| 20| 50| 34|Accelerometer-201...|Brush_teeth|       6.0|
| 21| 49| 33|Accelerometer-201...|Brush_teeth|       6.0|
| 21| 49| 33|Accelerometer-201...|Brush_teeth|       6.0|
| 20| 51| 35|A

In [4]:
from pyspark.ml.feature import OneHotEncoder, StringIndexer

encoder = OneHotEncoder(inputCol="classIndex", outputCol="categoryVec")
encoded = encoder.transform(indexed)
encoded.show()

+---+---+---+--------------------+-----------+----------+--------------+
|  x|  y|  z|              source|      class|classIndex|   categoryVec|
+---+---+---+--------------------+-----------+----------+--------------+
| 22| 49| 35|Accelerometer-201...|Brush_teeth|       6.0|(13,[6],[1.0])|
| 22| 49| 35|Accelerometer-201...|Brush_teeth|       6.0|(13,[6],[1.0])|
| 22| 52| 35|Accelerometer-201...|Brush_teeth|       6.0|(13,[6],[1.0])|
| 22| 52| 35|Accelerometer-201...|Brush_teeth|       6.0|(13,[6],[1.0])|
| 21| 52| 34|Accelerometer-201...|Brush_teeth|       6.0|(13,[6],[1.0])|
| 22| 51| 34|Accelerometer-201...|Brush_teeth|       6.0|(13,[6],[1.0])|
| 20| 50| 35|Accelerometer-201...|Brush_teeth|       6.0|(13,[6],[1.0])|
| 22| 52| 34|Accelerometer-201...|Brush_teeth|       6.0|(13,[6],[1.0])|
| 22| 50| 34|Accelerometer-201...|Brush_teeth|       6.0|(13,[6],[1.0])|
| 22| 51| 35|Accelerometer-201...|Brush_teeth|       6.0|(13,[6],[1.0])|
| 21| 51| 33|Accelerometer-201...|Brush_teeth|     

In [5]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

vectorAssembler = VectorAssembler(inputCols=["x","y","z"], outputCol="features")

# For your special case that has string instead of doubles you should cast them first.
# expr = [col(c).cast("Double").alias(c) 
#         for c in vectorAssembler.getInputCols()]

# df2 = df2.select(*expr)
features_vectorized = vectorAssembler.transform(encoded)
features_vectorized.show()

+---+---+---+--------------------+-----------+----------+--------------+----------------+
|  x|  y|  z|              source|      class|classIndex|   categoryVec|        features|
+---+---+---+--------------------+-----------+----------+--------------+----------------+
| 22| 49| 35|Accelerometer-201...|Brush_teeth|       6.0|(13,[6],[1.0])|[22.0,49.0,35.0]|
| 22| 49| 35|Accelerometer-201...|Brush_teeth|       6.0|(13,[6],[1.0])|[22.0,49.0,35.0]|
| 22| 52| 35|Accelerometer-201...|Brush_teeth|       6.0|(13,[6],[1.0])|[22.0,52.0,35.0]|
| 22| 52| 35|Accelerometer-201...|Brush_teeth|       6.0|(13,[6],[1.0])|[22.0,52.0,35.0]|
| 21| 52| 34|Accelerometer-201...|Brush_teeth|       6.0|(13,[6],[1.0])|[21.0,52.0,34.0]|
| 22| 51| 34|Accelerometer-201...|Brush_teeth|       6.0|(13,[6],[1.0])|[22.0,51.0,34.0]|
| 20| 50| 35|Accelerometer-201...|Brush_teeth|       6.0|(13,[6],[1.0])|[20.0,50.0,35.0]|
| 22| 52| 34|Accelerometer-201...|Brush_teeth|       6.0|(13,[6],[1.0])|[22.0,52.0,34.0]|
| 22| 50| 

In [6]:
from pyspark.ml.feature import MinMaxScaler
from pyspark.ml.linalg import Vectors

min_max_scaler = MinMaxScaler(inputCol="features", outputCol="features_norm")
min_max_scaler_model = min_max_scaler.fit(features_vectorized)
normalized_data = min_max_scaler_model.transform(features_vectorized)

normalized_data.show()

+---+---+---+--------------------+-----------+----------+--------------+----------------+--------------------+
|  x|  y|  z|              source|      class|classIndex|   categoryVec|        features|       features_norm|
+---+---+---+--------------------+-----------+----------+--------------+----------------+--------------------+
| 22| 49| 35|Accelerometer-201...|Brush_teeth|       6.0|(13,[6],[1.0])|[22.0,49.0,35.0]|[0.34920634920634...|
| 22| 49| 35|Accelerometer-201...|Brush_teeth|       6.0|(13,[6],[1.0])|[22.0,49.0,35.0]|[0.34920634920634...|
| 22| 52| 35|Accelerometer-201...|Brush_teeth|       6.0|(13,[6],[1.0])|[22.0,52.0,35.0]|[0.34920634920634...|
| 22| 52| 35|Accelerometer-201...|Brush_teeth|       6.0|(13,[6],[1.0])|[22.0,52.0,35.0]|[0.34920634920634...|
| 21| 52| 34|Accelerometer-201...|Brush_teeth|       6.0|(13,[6],[1.0])|[21.0,52.0,34.0]|[0.33333333333333...|
| 22| 51| 34|Accelerometer-201...|Brush_teeth|       6.0|(13,[6],[1.0])|[22.0,51.0,34.0]|[0.34920634920634...|
|

In [9]:
df_train = normalized_data.drop("source")\
  .drop("class")\
  .drop("classIndex")\
  .drop("features")\
  .drop("x")\
  .drop("y")\
  .drop("z")

In [10]:
df_train.show()

+--------------+--------------------+
|   categoryVec|       features_norm|
+--------------+--------------------+
|(13,[6],[1.0])|[0.34920634920634...|
|(13,[6],[1.0])|[0.34920634920634...|
|(13,[6],[1.0])|[0.34920634920634...|
|(13,[6],[1.0])|[0.34920634920634...|
|(13,[6],[1.0])|[0.33333333333333...|
|(13,[6],[1.0])|[0.34920634920634...|
|(13,[6],[1.0])|[0.31746031746031...|
|(13,[6],[1.0])|[0.34920634920634...|
|(13,[6],[1.0])|[0.34920634920634...|
|(13,[6],[1.0])|[0.34920634920634...|
|(13,[6],[1.0])|[0.33333333333333...|
|(13,[6],[1.0])|[0.31746031746031...|
|(13,[6],[1.0])|[0.33333333333333...|
|(13,[6],[1.0])|[0.33333333333333...|
|(13,[6],[1.0])|[0.31746031746031...|
|(13,[6],[1.0])|[0.28571428571428...|
|(13,[6],[1.0])|[0.30158730158730...|
|(13,[6],[1.0])|[0.25396825396825...|
|(13,[6],[1.0])|[0.28571428571428...|
|(13,[6],[1.0])|[0.28571428571428...|
+--------------+--------------------+
only showing top 20 rows



In [11]:
from pyspark.ml import Pipeline

pipeline = Pipeline(stages=[indexer, encoder, vectorAssembler, min_max_scaler_model])
model = pipeline.fit(df)
prediction = model.transform(df)

In [12]:
prediction.show()

+---+---+---+--------------------+-----------+----------+--------------+----------------+--------------------+
|  x|  y|  z|              source|      class|classIndex|   categoryVec|        features|       features_norm|
+---+---+---+--------------------+-----------+----------+--------------+----------------+--------------------+
| 22| 49| 35|Accelerometer-201...|Brush_teeth|       6.0|(13,[6],[1.0])|[22.0,49.0,35.0]|[0.34920634920634...|
| 22| 49| 35|Accelerometer-201...|Brush_teeth|       6.0|(13,[6],[1.0])|[22.0,49.0,35.0]|[0.34920634920634...|
| 22| 52| 35|Accelerometer-201...|Brush_teeth|       6.0|(13,[6],[1.0])|[22.0,52.0,35.0]|[0.34920634920634...|
| 22| 52| 35|Accelerometer-201...|Brush_teeth|       6.0|(13,[6],[1.0])|[22.0,52.0,35.0]|[0.34920634920634...|
| 21| 52| 34|Accelerometer-201...|Brush_teeth|       6.0|(13,[6],[1.0])|[21.0,52.0,34.0]|[0.33333333333333...|
| 22| 51| 34|Accelerometer-201...|Brush_teeth|       6.0|(13,[6],[1.0])|[22.0,51.0,34.0]|[0.34920634920634...|
|